In [1]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# --- 日本語設定 ---
plt.rcParams['font.family'] = 'Meiryo'
plt.rcParams['axes.unicode_minus'] = False
# ------------------

# ファイルの読み込みとデータ準備
df = pd.read_csv("bikecycle.csv")
df['発生年月日（始期）'] = pd.to_datetime(df['発生年月日（始期）'], format='%Y%m%d', errors='coerce')
df.dropna(subset=['発生年月日（始期）'], inplace=True)
df['市区町村'] = df['市区町村（発生地）']
df['完全住所'] = '千葉県' + df['市区町村（発生地）'] 

# 市区町村ごとの代表座標を格納する辞書
city_coords = {}
geolocator = Nominatim(user_agent="bicycle_theft_analysis")

# --- 座標の取得（ジオコーディング） ---
print("地図プロットのための座標を確認または再取得中です...")
for city in df['市区町村（発生地）'].unique():
    full_city_name = '千葉県' + city
    if full_city_name not in city_coords:
        try:
            location = geolocator.geocode(full_city_name, timeout=10)
            if location:
                city_coords[full_city_name] = (location.latitude, location.longitude)
            else:
                city_coords[full_city_name] = None
        except:
            city_coords[full_city_name] = None
        time.sleep(0.1) # 負荷軽減のため待機
# ----------------------------------------

# --- 地図の作成とプロットに必要な準備 ---
city_counts = df.groupby('市区町村（発生地）').size().reset_index(name='件数')
city_counts['完全住所'] = '千葉県' + city_counts['市区町村（発生地）'] 

min_count = city_counts['件数'].min()
max_count = city_counts['件数'].max()
cmap = plt.cm.get_cmap('RdYlBu_r') 
norm = mcolors.Normalize(vmin=min_count, vmax=max_count)

# 地図を初期化
map_center = [35.6047, 140.1238] 
m = folium.Map(location=map_center, zoom_start=10)

# プロットの実行 (地図上のマーカーは維持)
for index, row in city_counts.iterrows():
    full_address = row['完全住所']
    count = row['件数']
    
    if full_address in city_coords and city_coords[full_address] is not None:
        lat, lon = city_coords[full_address]
        rgba_color = cmap(norm(count))
        hex_color = mcolors.rgb2hex(rgba_color)
        
        popup_html = f"""
        <div style="writing-mode: horizontal-tb; font-family: Meiryo, sans-serif;">
            📍 <b>{row['市区町村（発生地）']}</b><br>
            盗難件数: {count} 件
        </div>
        """
        
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=hex_color,
            fill=True,
            fill_color=hex_color,
            fill_opacity=0.8,
            popup=folium.Popup(popup_html, max_width=200)
        ).add_to(m)

# ----------------------------------------------------
# 1. 上位ランキングの追加 (左上)
# ----------------------------------------------------
city_ranking_top = df.groupby('市区町村（発生地）').size().reset_index(name='件数').sort_values(by='件数', ascending=False).reset_index(drop=True)
city_ranking_top = city_ranking_top.rename(columns={'市区町村（発生地）': '市区町村'})

ranking_html_top = ""
for i, row in city_ranking_top.head(15).iterrows():
    count = row['件数']
    rank_number = i + 1 
    rgba_color = cmap(norm(count))
    hex_color = mcolors.rgb2hex(rgba_color)
    
    ranking_html_top += f"""
    <tr>
        <td style="text-align: right; width: 10%;">{rank_number}位</td>
        <td style="width: 10%;"><span style="color: {hex_color}; font-size: 20px;">●</span></td>
        <td style="width: 50%;">{row['市区町村']}</td>
        <td style="text-align: right; font-weight: bold; width: 30%;">{count}</td>
    </tr>
    """

# ランキング全体を囲むHTML (左上: top: 10px, left: 10px)
ranking_table_top_html = f"""
<div style="position: fixed; 
            top: 10px; left: 10px; width: 280px; height: auto; max-height: 45vh; 
            border: 2px solid #555; z-index: 9999; font-size: 14px;
            background-color: white; opacity: 0.95; padding: 10px; 
            overflow-y: auto; box-shadow: 3px 3px 5px rgba(0,0,0,0.3); font-family: Meiryo, sans-serif;">
    <h4 style="margin: 0 0 10px 0; border-bottom: 1px solid #ccc;">🚨 上位 盗難件数ランキング (多い順)</h4>
    <table style="width: 100%; border-collapse: collapse;">
        <thead>
            <tr style="border-bottom: 1px solid #ccc;">
                <th style="text-align: right; padding-bottom: 5px;">順位</th>
                <th></th>
                <th style="text-align: left; padding-bottom: 5px;">市区町村</th>
                <th style="text-align: right; padding-bottom: 5px;">件数</th>
            </tr>
        </thead>
        <tbody>
            {ranking_html_top}
        </tbody>
    </table>
</div>
"""
m.get_root().html.add_child(folium.Element(ranking_table_top_html))

# ----------------------------------------------------
# 2. 下位ランキングの追加 (左側、上位ランキングの下)
# ----------------------------------------------------

# 下位ランキング (少ない順にソートし、インデックスをリセット)
city_ranking_bottom = df.groupby('市区町村（発生地）').size().reset_index(name='件数').sort_values(by='件数', ascending=True).reset_index(drop=True)
city_ranking_bottom = city_ranking_bottom.rename(columns={'市区町村（発生地）': '市区町村'})

ranking_html_bottom = ""
for i, row in city_ranking_bottom.head(15).iterrows():
    count = row['件数']
    rank_number = i + 1 
    rgba_color = cmap(norm(count))
    hex_color = mcolors.rgb2hex(rgba_color)
    
    ranking_html_bottom += f"""
    <tr>
        <td style="text-align: right; width: 10%;">{rank_number}位</td>
        <td style="width: 10%;"><span style="color: {hex_color}; font-size: 20px;">●</span></td>
        <td style="width: 50%;">{row['市区町村']}</td>
        <td style="text-align: right; font-weight: bold; width: 30%;">{count}</td>
    </tr>
    """

# ランキング全体を囲むHTML (左側、下位に配置: top: 50vh + 20px)
ranking_table_bottom_html = f"""
<div style="position: fixed; 
            top: 50vh; left: 10px; width: 280px; height: auto; max-height: 45vh; 
            border: 2px solid #555; z-index: 9999; font-size: 14px;
            background-color: white; opacity: 0.95; padding: 10px; 
            overflow-y: auto; box-shadow: 3px 3px 5px rgba(0,0,0,0.3); font-family: Meiryo, sans-serif;">
    <h4 style="margin: 0 0 10px 0; border-bottom: 1px solid #ccc;">🟢 下位 盗難件数ランキング (少ない順)</h4>
    <table style="width: 100%; border-collapse: collapse;">
        <thead>
            <tr style="border-bottom: 1px solid #ccc;">
                <th style="text-align: right; padding-bottom: 5px;">順位</th>
                <th></th>
                <th style="text-align: left; padding-bottom: 5px;">市区町村</th>
                <th style="text-align: right; padding-bottom: 5px;">件数</th>
            </tr>
        </thead>
        <tbody>
            {ranking_html_bottom}
        </tbody>
    </table>
</div>
"""
m.get_root().html.add_child(folium.Element(ranking_table_bottom_html))

# ----------------------------------------------------
# 3. 凡例 (カラーバー) の削除 (処理をスキップ)
# ----------------------------------------------------
# 以前の凡例HTMLの追加処理は削除されました。

print("\nランキングを左側に上下に配置し、凡例を削除した地図の作成を完了しました。")

# VScode環境でHTMLとして保存し、表示
map_filename = 'bicycle_theft_map_with_both_ranking.html'
m.save(map_filename)
print(f"地図ファイルが'{map_filename}'として上書き保存されました。ブラウザで開いて確認してください。")

地図プロットのための座標を確認または再取得中です...

ランキングを左側に上下に配置し、凡例を削除した地図の作成を完了しました。
地図ファイルが'bicycle_theft_map_with_both_ranking.html'として上書き保存されました。ブラウザで開いて確認してください。


C:\Users\gatai\AppData\Local\Temp\ipykernel_24656\1841772762.py:46: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('RdYlBu_r')


In [2]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json # 座標データをJSON形式で埋め込むために使用

# --- 日本語設定 ---
plt.rcParams['font.family'] = 'Meiryo'
plt.rcParams['axes.unicode_minus'] = False
# ------------------

# ファイルの読み込みとデータ準備
df = pd.read_csv("bikecycle.csv")
df['発生年月日（始期）'] = pd.to_datetime(df['発生年月日（始期）'], format='%Y%m%d', errors='coerce')
df.dropna(subset=['発生年月日（始期）'], inplace=True)
df['市区町村'] = df['市区町村（発生地）']
df['完全住所'] = '千葉県' + df['市区町村（発生地）'] 

# 市区町村ごとの代表座標を格納する辞書
city_coords = {}
geolocator = Nominatim(user_agent="bicycle_theft_analysis")

# --- 座標の取得（ジオコーディング） ---
print("地図プロットのための座標を確認または再取得中です...")
for city in df['市区町村（発生地）'].unique():
    full_city_name = '千葉県' + city
    if full_city_name not in city_coords:
        try:
            location = geolocator.geocode(full_city_name, timeout=10)
            if location:
                city_coords[full_city_name] = (location.latitude, location.longitude)
            else:
                city_coords[full_city_name] = None
        except:
            city_coords[full_city_name] = None
        time.sleep(0.1)
# ----------------------------------------

# --- 地図の作成とプロットに必要な準備 ---
city_counts = df.groupby('市区町村（発生地）').size().reset_index(name='件数')
city_counts['完全住所'] = '千葉県' + city_counts['市区町村（発生地）'] 

min_count = city_counts['件数'].min()
max_count = city_counts['件数'].max()
cmap = plt.cm.get_cmap('RdYlBu_r') 
norm = mcolors.Normalize(vmin=min_count, vmax=max_count)

# 座標をJavaScriptで使いやすい形式に変換: {'千葉県A市': [lat, lon], ...}
js_coords = {full_name: list(coords) for full_name, coords in city_coords.items() if coords is not None}
js_coords_json = json.dumps(js_coords)

# 地図を初期化
map_center = [35.6047, 140.1238] 
m = folium.Map(location=map_center, zoom_start=10)

# --- 1. JavaScript関数の追加 ---
js_code = f"""
var cityCoords = {js_coords_json};
var map = L.getBounds(null).getCenter(); // 仮の地図オブジェクト取得

// 地図オブジェクトが利用可能になったら、変数に代入
m.on('load', function() {{
    map = this;
}});

function zoomToCity(fullCityName) {{
    var coords = cityCoords[fullCityName];
    if (coords && map) {{
        // L.Map.setView(LatLng, zoom) を使用
        map.setView(new L.LatLng(coords[0], coords[1]), 13); // ズームレベルを13に設定
        
        // ポップアップを自動で開く (オプション: 今回は実装しない)
        /*
        m.eachLayer(function(layer) {{
            if (layer.options && layer.options.popup && layer.options.popup.options.content.includes(fullCityName)) {{
                layer.openPopup();
            }}
        }});
        */
    }} else {{
        console.log("座標が見つからないか、地図がロードされていません: " + fullCityName);
    }}
}}
"""
m.get_root().script.add_child(folium.Element(js_code))


# プロットの実行 (地図上のマーカー)
for index, row in city_counts.iterrows():
    full_address = row['完全住所']
    count = row['件数']
    
    if full_address in city_coords and city_coords[full_address] is not None:
        lat, lon = city_coords[full_address]
        rgba_color = cmap(norm(count))
        hex_color = mcolors.rgb2hex(rgba_color)
        
        popup_html = f"""
        <div style="writing-mode: horizontal-tb; font-family: Meiryo, sans-serif;">
            📍 <b>{row['市区町村（発生地）']}</b><br>
            盗難件数: {count} 件
        </div>
        """
        
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=hex_color,
            fill=True,
            fill_color=hex_color,
            fill_opacity=0.8,
            popup=folium.Popup(popup_html, max_width=200)
        ).add_to(m)

# ----------------------------------------------------
# 2. ランキング HTML の生成 (★onclickを追加★)
# ----------------------------------------------------

# --- 上位ランキング ---
city_ranking_top = df.groupby('市区町村（発生地）').size().reset_index(name='件数').sort_values(by='件数', ascending=False).reset_index(drop=True)
city_ranking_top['完全住所'] = '千葉県' + city_ranking_top['市区町村（発生地）'] # クリック用に完全住所を付与
city_ranking_top = city_ranking_top.rename(columns={'市区町村（発生地）': '市区町村'})

ranking_html_top = ""
for i, row in city_ranking_top.head(15).iterrows():
    count = row['件数']
    rank_number = i + 1 
    rgba_color = cmap(norm(count))
    hex_color = mcolors.rgb2hex(rgba_color)
    
    # ★修正点: クリックイベントを追加
    onclick_event = f"zoomToCity('{row['完全住所']}')"
    
    ranking_html_top += f"""
    <tr onclick="{onclick_event}" style="cursor: pointer;" title="クリックで地図を移動">
        <td style="text-align: right; width: 10%;">{rank_number}位</td>
        <td style="width: 10%;"><span style="color: {hex_color}; font-size: 20px;">●</span></td>
        <td style="width: 50%; text-decoration: underline;">{row['市区町村']}</td>
        <td style="text-align: right; font-weight: bold; width: 30%;">{count}</td>
    </tr>
    """

# ランキング全体を囲むHTML (左上)
ranking_table_top_html = f"""
<div style="position: fixed; 
            top: 10px; left: 10px; width: 280px; height: auto; max-height: 45vh; 
            border: 2px solid #555; z-index: 9999; font-size: 14px;
            background-color: white; opacity: 0.95; padding: 10px; 
            overflow-y: auto; box-shadow: 3px 3px 5px rgba(0,0,0,0.3); font-family: Meiryo, sans-serif;">
    <h4 style="margin: 0 0 10px 0; border-bottom: 1px solid #ccc;">🚨 上位 盗難件数ランキング (多い順)</h4>
    <table style="width: 100%; border-collapse: collapse;">
        <thead>
            <tr style="border-bottom: 1px solid #ccc;">
                <th style="text-align: right; padding-bottom: 5px;">順位</th>
                <th></th>
                <th style="text-align: left; padding-bottom: 5px;">市区町村</th>
                <th style="text-align: right; padding-bottom: 5px;">件数</th>
            </tr>
        </thead>
        <tbody>
            {ranking_html_top}
        </tbody>
    </table>
</div>
"""
m.get_root().html.add_child(folium.Element(ranking_table_top_html))

# --- 下位ランキング ---
city_ranking_bottom = df.groupby('市区町村（発生地）').size().reset_index(name='件数').sort_values(by='件数', ascending=True).reset_index(drop=True)
city_ranking_bottom['完全住所'] = '千葉県' + city_ranking_bottom['市区町村（発生地）'] # クリック用に完全住所を付与
city_ranking_bottom = city_ranking_bottom.rename(columns={'市区町村（発生地）': '市区町村'})

ranking_html_bottom = ""
for i, row in city_ranking_bottom.head(15).iterrows():
    count = row['件数']
    rank_number = i + 1 
    rgba_color = cmap(norm(count))
    hex_color = mcolors.rgb2hex(rgba_color)
    
    # ★修正点: クリックイベントを追加
    onclick_event = f"zoomToCity('{row['完全住所']}')"
    
    ranking_html_bottom += f"""
    <tr onclick="{onclick_event}" style="cursor: pointer;" title="クリックで地図を移動">
        <td style="text-align: right; width: 10%;">{rank_number}位</td>
        <td style="width: 10%;"><span style="color: {hex_color}; font-size: 20px;">●</span></td>
        <td style="width: 50%; text-decoration: underline;">{row['市区町村']}</td>
        <td style="text-align: right; font-weight: bold; width: 30%;">{count}</td>
    </tr>
    """

# ランキング全体を囲むHTML (左側、下位に配置)
ranking_table_bottom_html = f"""
<div style="position: fixed; 
            top: 50vh; left: 10px; width: 280px; height: auto; max-height: 45vh; 
            border: 2px solid #555; z-index: 9999; font-size: 14px;
            background-color: white; opacity: 0.95; padding: 10px; 
            overflow-y: auto; box-shadow: 3px 3px 5px rgba(0,0,0,0.3); font-family: Meiryo, sans-serif;">
    <h4 style="margin: 0 0 10px 0; border-bottom: 1px solid #ccc;">🟢 下位 盗難件数ランキング (少ない順)</h4>
    <table style="width: 100%; border-collapse: collapse;">
        <thead>
            <tr style="border-bottom: 1px solid #ccc;">
                <th style="text-align: right; padding-bottom: 5px;">順位</th>
                <th></th>
                <th style="text-align: left; padding-bottom: 5px;">市区町村</th>
                <th style="text-align: right; padding-bottom: 5px;">件数</th>
            </tr>
        </thead>
        <tbody>
            {ranking_html_bottom}
        </tbody>
    </table>
</div>
"""
m.get_root().html.add_child(folium.Element(ranking_table_bottom_html))

print("\nランキングクリックで地図が移動する機能を追加しました。")

# VScode環境でHTMLとして保存し、表示
map_filename = 'bicycle_theft_map_with_both_ranking.html'
m.save(map_filename)
print(f"地図ファイルが'{map_filename}'として上書き保存されました。ブラウザで開いて確認してください。")

地図プロットのための座標を確認または再取得中です...

ランキングクリックで地図が移動する機能を追加しました。
地図ファイルが'bicycle_theft_map_with_both_ranking.html'として上書き保存されました。ブラウザで開いて確認してください。


C:\Users\gatai\AppData\Local\Temp\ipykernel_24656\2428126235.py:47: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('RdYlBu_r')


In [3]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json 
import branca.element as be # JavaScriptを地図に埋め込むために必要

# --- 日本語設定 ---
plt.rcParams['font.family'] = 'Meiryo'
plt.rcParams['axes.unicode_minus'] = False
# ------------------

# ファイルの読み込みとデータ準備
df = pd.read_csv("bikecycle.csv")
df['発生年月日（始期）'] = pd.to_datetime(df['発生年月日（始期）'], format='%Y%m%d', errors='coerce')
df.dropna(subset=['発生年月日（始期）'], inplace=True)
df['市区町村'] = df['市区町村（発生地）']
df['完全住所'] = '千葉県' + df['市区町村（発生地）'] 

# 市区町村ごとの代表座標を格納する辞書
city_coords = {}
geolocator = Nominatim(user_agent="bicycle_theft_analysis")

# --- 座標の取得（ジオコーディング） ---
print("地図プロットのための座標を確認または再取得中です...")
for city in df['市区町村（発生地）'].unique():
    full_city_name = '千葉県' + city
    if full_city_name not in city_coords:
        try:
            location = geolocator.geocode(full_city_name, timeout=10)
            if location:
                city_coords[full_city_name] = (location.latitude, location.longitude)
            else:
                city_coords[full_city_name] = None
        except:
            city_coords[full_city_name] = None
        time.sleep(0.1)
# ----------------------------------------

# --- 地図の作成とプロットに必要な準備 ---
city_counts = df.groupby('市区町村（発生地）').size().reset_index(name='件数')
city_counts['完全住所'] = '千葉県' + city_counts['市区町村（発生地）'] 

min_count = city_counts['件数'].min()
max_count = city_counts['件数'].max()
cmap = plt.cm.get_cmap('RdYlBu_r') 
norm = mcolors.Normalize(vmin=min_count, vmax=max_count)

# 座標をJavaScriptで使いやすい形式に変換
js_coords = {full_name: list(coords) for full_name, coords in city_coords.items() if coords is not None}
js_coords_json = json.dumps(js_coords)

# 地図を初期化
map_center = [35.6047, 140.1238] 
m = folium.Map(location=map_center, zoom_start=10)

# --- 1. JavaScript関数の追加 (地図オブジェクトを参照する処理を修正) ---
js_code = f"""
var cityCoords = {js_coords_json};
var map = null; 

// Foliumの地図オブジェクト（m）がLeafletマップオブジェクトを参照するのを待つための処理
// m は FoliumのPython変数名であり、HTML/JS上では `window.m` になることが期待される
setTimeout(function() {{
    // 実際にFoliumが生成したLeafletマップインスタンスを取得
    // Folium 0.12以降、地図オブジェクトは要素IDに基づいてグローバル変数としてアクセス可能
    // ただし確実性を高めるため、setTimeoutで少し遅らせて参照を試みる
    if (window.m) {{
        map = window.m;
    }} else {{
        // 代替策: グローバルスコープからLeafletマップインスタンスを検索
        for (var key in window) {{
            if (window[key] && window[key]._leaflet_id && window[key].setView) {{
                map = window[key];
                break;
            }}
        }}
    }}
}}, 300); // 300ミリ秒待機

function zoomToCity(fullCityName) {{
    var coords = cityCoords[fullCityName];
    if (coords && map) {{
        // Leafletの setView(LatLng, zoom) を使用
        map.setView([coords[0], coords[1]], 13); // ズームレベルは13に固定
    }} else {{
        console.log("座標が見つからないか、地図がロードされていません: " + fullCityName);
    }}
}}
"""
# branca.element.Element で <script> タグとして挿入
m.get_root().script.add_child(be.Element(js_code))


# プロットの実行 (地図上のマーカー)
for index, row in city_counts.iterrows():
    full_address = row['完全住所']
    count = row['件数']
    
    if full_address in city_coords and city_coords[full_address] is not None:
        lat, lon = city_coords[full_address]
        rgba_color = cmap(norm(count))
        hex_color = mcolors.rgb2hex(rgba_color)
        
        popup_html = f"""
        <div style="writing-mode: horizontal-tb; font-family: Meiryo, sans-serif;">
            📍 <b>{row['市区町村（発生地）']}</b><br>
            盗難件数: {count} 件
        </div>
        """
        
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color=hex_color,
            fill=True,
            fill_color=hex_color,
            fill_opacity=0.8,
            popup=folium.Popup(popup_html, max_width=200)
        ).add_to(m)

# ----------------------------------------------------
# 2. ランキング HTML の生成 (左側に上下配置)
# ----------------------------------------------------

# --- 上位ランキング (左上) ---
city_ranking_top = df.groupby('市区町村（発生地）').size().reset_index(name='件数').sort_values(by='件数', ascending=False).reset_index(drop=True)
city_ranking_top['完全住所'] = '千葉県' + city_ranking_top['市区町村（発生地）'] 
city_ranking_top = city_ranking_top.rename(columns={'市区町村（発生地）': '市区町村'})

ranking_html_top = ""
for i, row in city_ranking_top.head(15).iterrows():
    count = row['件数']
    rank_number = i + 1 
    rgba_color = cmap(norm(count))
    hex_color = mcolors.rgb2hex(rgba_color)
    
    # クリックイベント
    onclick_event = f"zoomToCity('{row['完全住所']}')"
    
    ranking_html_top += f"""
    <tr onclick="{onclick_event}" style="cursor: pointer;" title="クリックで地図を移動">
        <td style="text-align: right; width: 10%;">{rank_number}位</td>
        <td style="width: 10%;"><span style="color: {hex_color}; font-size: 20px;">●</span></td>
        <td style="width: 50%; text-decoration: underline;">{row['市区町村']}</td>
        <td style="text-align: right; font-weight: bold; width: 30%;">{count}</td>
    </tr>
    """

# ランキング全体を囲むHTML (左上: top: 10px, left: 10px)
ranking_table_top_html = f"""
<div style="position: fixed; 
            top: 10px; left: 10px; width: 280px; height: auto; max-height: 45vh; 
            border: 2px solid #555; z-index: 9999; font-size: 14px;
            background-color: white; opacity: 0.95; padding: 10px; 
            overflow-y: auto; box-shadow: 3px 3px 5px rgba(0,0,0,0.3); font-family: Meiryo, sans-serif;">
    <h4 style="margin: 0 0 10px 0; border-bottom: 1px solid #ccc;">🚨 上位 盗難件数ランキング (多い順)</h4>
    <table style="width: 100%; border-collapse: collapse;">
        <thead>
            <tr style="border-bottom: 1px solid #ccc;">
                <th style="text-align: right; padding-bottom: 5px;">順位</th>
                <th></th>
                <th style="text-align: left; padding-bottom: 5px;">市区町村</th>
                <th style="text-align: right; padding-bottom: 5px;">件数</th>
            </tr>
        </thead>
        <tbody>
            {ranking_html_top}
        </tbody>
    </table>
</div>
"""
m.get_root().html.add_child(folium.Element(ranking_table_top_html))

# --- 下位ランキング (左下) ---
city_ranking_bottom = df.groupby('市区町村（発生地）').size().reset_index(name='件数').sort_values(by='件数', ascending=True).reset_index(drop=True)
city_ranking_bottom['完全住所'] = '千葉県' + city_ranking_bottom['市区町村（発生地）']
city_ranking_bottom = city_ranking_bottom.rename(columns={'市区町村（発生地）': '市区町村'})

ranking_html_bottom = ""
for i, row in city_ranking_bottom.head(15).iterrows():
    count = row['件数']
    rank_number = i + 1 
    rgba_color = cmap(norm(count))
    hex_color = mcolors.rgb2hex(rgba_color)
    
    # クリックイベント
    onclick_event = f"zoomToCity('{row['完全住所']}')"
    
    ranking_html_bottom += f"""
    <tr onclick="{onclick_event}" style="cursor: pointer;" title="クリックで地図を移動">
        <td style="text-align: right; width: 10%;">{rank_number}位</td>
        <td style="width: 10%;"><span style="color: {hex_color}; font-size: 20px;">●</span></td>
        <td style="width: 50%; text-decoration: underline;">{row['市区町村']}</td>
        <td style="text-align: right; font-weight: bold; width: 30%;">{count}</td>
    </tr>
    """

# ランキング全体を囲むHTML (左側、下位に配置: top: 50vh)
ranking_table_bottom_html = f"""
<div style="position: fixed; 
            top: 50vh; left: 10px; width: 280px; height: auto; max-height: 45vh; 
            border: 2px solid #555; z-index: 9999; font-size: 14px;
            background-color: white; opacity: 0.95; padding: 10px; 
            overflow-y: auto; box-shadow: 3px 3px 5px rgba(0,0,0,0.3); font-family: Meiryo, sans-serif;">
    <h4 style="margin: 0 0 10px 0; border-bottom: 1px solid #ccc;">🟢 下位 盗難件数ランキング (少ない順)</h4>
    <table style="width: 100%; border-collapse: collapse;">
        <thead>
            <tr style="border-bottom: 1px solid #ccc;">
                <th style="text-align: right; padding-bottom: 5px;">順位</th>
                <th></th>
                <th style="text-align: left; padding-bottom: 5px;">市区町村</th>
                <th style="text-align: right; padding-bottom: 5px;">件数</th>
            </tr>
        </thead>
        <tbody>
            {ranking_html_bottom}
        </tbody>
    </table>
</div>
"""
m.get_root().html.add_child(folium.Element(ranking_table_bottom_html))

print("\n地図が表示されない問題を修正し、ランキングクリックで地図が移動する機能を追加しました。")

# VScode環境でHTMLとして保存し、表示
map_filename = 'bicycle_theft_map_with_both_ranking.html'
m.save(map_filename)
print(f"地図ファイルが'{map_filename}'として上書き保存されました。ブラウザで開いて確認してください。")

地図プロットのための座標を確認または再取得中です...

地図が表示されない問題を修正し、ランキングクリックで地図が移動する機能を追加しました。
地図ファイルが'bicycle_theft_map_with_both_ranking.html'として上書き保存されました。ブラウザで開いて確認してください。


C:\Users\gatai\AppData\Local\Temp\ipykernel_24656\2792245501.py:48: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('RdYlBu_r')
